<a href="https://colab.research.google.com/github/RuthlessActuary2023/CAS_RPM_2023/blob/main/2023_CAS_RPM_XGB_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2023 CAS RPM GBM
### Regression using XGBoost

### Install Required Packages

In [1]:
install.packages('dplyr')
install.packages('tidyverse')
install.packages('xgboost')
install.packages('ggplot2')
install.packages('plotly')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘textshaping’, ‘conflicted’, ‘jsonlite’, ‘ragg’, ‘stringr’


Warning message in install.packages("tidyverse"):
“installation of package ‘textshaping’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘ragg’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘tidyverse’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘later’, ‘htmlwidgets’, ‘lazyeval’, ‘crosstalk’, ‘promises’




### Load Required Packages

In [2]:
require(dplyr)
require(tidyverse)
require(xgboost)
library(ggplot2)
library(plotly)

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ stringr 1.5.0
✔ tidyr   1.3.0     ✔ forcats 1.0.0
✔ readr   2.1.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: xgboost


Attaching package: ‘xgboost’


The following object is masked from ‘package:dplyr’:

    slice



Attaching package: ‘plotly’


The following object is masked from ‘package:xgboost’:

    slice


The following object is 

### Define Functions

In [3]:
## resid.plot includes an off balance for when the model overall predicts low or high
resid.plot <- function(data,var,actual,pred,weight,rt_text,lt_text) {
  pr1 = paste0(paste0(pred,'*'),weight)
  
  header = "Actual vs Predicted; Actual Values"
  
  home_plot <- data %>%
    mutate(nt = eval(parse(text = var))) %>%
    group_by(nt) %>%
    summarize(  act = sum(eval(parse(text = actual)))
                , pred = sum(eval(parse(text = pr1)))
                , exposures = sum(eval(parse(text = weight)))
    ) %>%
    mutate(   actual = act/exposures
              , predicted = pred/exposures)
  
  home_plot <- home_plot[complete.cases(home_plot),]
  
  # off balance for model total prediction
  a1 = paste0("data$",actual)
  a2 = paste0("data$",pred)
  a3 = paste0("data$",weight)
  off_balance = sum(eval(parse(text = a1)))/sum(eval(parse(text = a2)) * eval(parse(text = a3)))
  home_plot$predicted <- home_plot$predicted*off_balance
  
  par(mar=c(5, 4, 2, 5) + 0.1)
  
  barplot(home_plot$exposures,las=2,col="lightblue",ylim = c(0,max(home_plot$exposures)*1.1))
  par(new=TRUE)
  plot(home_plot$nt, home_plot$actual, col='darkblue',lwd= 4, axes=FALSE, ylim=c(ifelse(min(home_plot$actual)<min(home_plot$predicted),min(home_plot$actual)*0.98,min(home_plot$predicted)*0.98),ifelse(max(home_plot$actual)>max(home_plot$predicted),max(home_plot$actual)*1.02,max(home_plot$predicted)*1.02)), xlab="", ylab="",main=header)
  axis(4, ylim=c(0,1),col="black",las=1)
  axis(1, xlim=c(0,1),col="black",las=1)
  mtext(rt_text,side=4,line=3.0,col="black",font=7)
  mtext(lt_text, side = 2, line=3.0,col="black",font=7)
  box()
  mtext(var,side=1,col="black",line=2.5,font=7) 
  lines(home_plot$nt,home_plot$predicted, col="green",lwd=3)  
}
## lift.chart does not include any off balance and creates a lift chart on test
lift.chart <- function() {
  lift1   <-  test %>%
    arrange(pred) %>%
    mutate(cs = cumsum(ee_pd))
  
  max_cs = max(lift1$cs)
  
  lift <- lift1 %>%
    mutate(nt = floor(10*cs/(max_cs+1)) + 1) %>%
    group_by(nt) %>%
    summarize(eexp = sum(ee_pd),
              act = sum(incurred_pd_cap),
              pred = sum(pred*ee_pd)
    ) %>%
    mutate(
      actual = act/eexp,
      predicted = pred/eexp
    )
  
  
  par(mar=c(5, 4, 4, 6) + 0.1)
  barplot(lift$eexp,las=2,col="lightblue",
          ylim = c(0,max(lift$eexp)*1.1))
  par(new=TRUE)
  plot(as.numeric(lift$nt), lift$actual, col='darkblue',lwd=4, axes=FALSE, ylim=c(min(lift$actual)*0.9,max(lift$actual)*1.25), xlab="", ylab="", 
       main="Pure Premium Lift Chart; PD")
  axis(4, ylim=c(0,1),col="black",las=1)  ## las=1 makes horizontal labels
  mtext("Pure Premium",side=4,line=3.0,col="black",font=7)
  mtext("Earned Exposures", side = 2, line=3.0,col="black",font=7)
  box()
  axis(1,lift$nt)
  mtext("Predicted Pure Premium",side=1,col="black",line=2.5,font=7) 
  lines(lift$nt,lift$predicted, col="green",lwd=3)
}

### Round 1 - Preliminary Fit with Basic Assumptions

In [4]:
# Import Data
path <- "https://raw.github.com/RuthlessActuary2023/CAS_RPM_2023/master/gbm_data_rpm_2023_1M_v3.zip"
temp <- tempfile()
download.file(path, temp)
unzip(temp, "gbm_data_rpm_2023_1M_v3.csv")
gbm_data <- read.csv("gbm_data_rpm_2023_1M_v3.csv")
unlink(temp)

gbm_data = subset(gbm_data, select = -c(rand) )
gbm_data$pp_pd <- gbm_data$incurred_pd_cap / gbm_data$ee_pd

# Split into train and test
size <- floor(0.80 * nrow(gbm_data))
set.seed(42)
train_idx <- sample(seq_len(nrow(gbm_data)), size = size)
train <- gbm_data[train_idx,]
test <- gbm_data[-train_idx,]

# Create Dmatrix and fit
exclude_fields <- names(train) %in% c("py","ee_pd","incurred_pd_cap","claim_count_pd","pp_pd")
train2 <- train[!exclude_fields]
test2 <- test[!exclude_fields]

dtrain <- xgb.DMatrix(data = as.matrix(train2)
                      , label = train$pp_pd
                      , weight = train$ee_pd
                      )
dtest <- xgb.DMatrix(data = as.matrix(test2)
                      , label = test$pp_pd
                      , weight = test$ee_pd
)

model <- xgboost(
  data = dtrain, 
  max.depth = 6, # max number of splits per tree
  eta = .05, #learning rate
  nthread = 4, # parallel threads
  nrounds = 50, # trees
  min_split_loss = 0, #minimum loss required to continue splitting
  min_child_weight = 0, #minimum weight required to continue splitting
  subsample = 1, #how much of train to use when building
  lambda = 1, #L2
  alpha = 0, #L1
  tree_method = 'hist', #approx, hist are other options for faster processing (use hist for parameter optimization)
  # monotone_constraints = ,# need to set this up
  base_score = sum(gbm_data$incurred_pd_cap)/sum(gbm_data$ee_pd),
  objective = "reg:tweedie",
  tweedie_variance_power = 1.5,
  verbose = 0
  )

train$pred <- predict(model,dtrain)
test$pred <- predict(model,dtest)

# Feature Importance & Residual Plots
importance_matrix = xgb.importance(colnames(dtrain), model = model)
importance_matrix = subset(importance_matrix, select = c(Feature,Gain))
lst = unique(importance_matrix[order(Gain),]$Feature) # Reverse order for plots

for (l in lst) {
  resid.plot(train,l,"incurred_pd_cap","pred","ee_pd","Pure Premium","Earned Exposure")
}
  
xgb.plot.importance(importance_matrix)

# Lift Chart
lift.chart()

ERROR: ignored